In [1]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')

    from google.colab import drive
    drive.mount('/content/drive')   

    import os
    os.chdir('/content/drive/My Drive/Deep_learning_project/')

else:
    print('Not running on CoLab')

Not running on CoLab


In [1]:
%load_ext autoreload
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.distributions import Normal
from torch.utils.data import TensorDataset, DataLoader

import datetime
import itertools

from tqdm import tqdm
import pickle
from torch.optim import lr_scheduler

from sklearn.model_selection import train_test_split

from modules import *

In [2]:
cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda else "cpu")

In [3]:
df = pd.read_csv('GM_preparedData.csv')

In [4]:
def format_tripStart(x, **kwargs):
    if hasattr(x, 'hour'):
        hour = x.hour * 60 * 60
        minute = x.minute * 60
        time_seconds = hour + minute
    else:
        x = datetime.timedelta(hours=x)
        time_seconds = x.seconds

    encoded_time = kwargs['encoder'](2 * np.pi * time_seconds / (24 * 60 * 60))
    
    return encoded_time


def format_time_split_hours(x):
    
    return int(x.hour)

In [5]:
# Define rectangle where "valid data belongs":

upper_right = (55.880120, 12.707644)
lower_left = (55.548626, 12.061736)

# Clean data
df_clean = df.loc[(df['startPositionLat'] >= lower_left[0]) & (df['startPositionLat']<=upper_right[0])]
df_clean = df_clean.loc[(df_clean['startPositionLng'] >= lower_left[1]) & (df_clean['startPositionLng']<=upper_right[1])]
df_clean = df_clean.dropna(axis=0, subset=['startPositionLat', 'startPositionLng', 'tripStart'])

df_clean['tripStart'] = pd.to_datetime(df_clean['tripStart'], format='%d%b%y:%H:%M:%S')
#df_clean['timeOfDay'] = df_clean['tripStart'].apply(format_tripStart)
df_clean['cos_timeOfDay'] = df_clean['tripStart'].apply(format_tripStart, encoder=np.cos)
df_clean['sin_timeOfDay'] = df_clean['tripStart'].apply(format_tripStart, encoder=np.sin)

df_clean['timeHour'] = df_clean['tripStart'].apply(format_time_split_hours)

In [6]:
df_clean = df_clean[['startPositionLat', 'startPositionLng', 'sin_timeOfDay', 'cos_timeOfDay', 'timeHour']]

In [8]:
data_columns = ['startPositionLng', 'startPositionLat', 'sin_timeOfDay', 'cos_timeOfDay']
data = df_clean[data_columns]

In [9]:
#Normalize data
train, test = train_test_split(data.values, test_size=0.1, random_state=42)
df_train = pd.DataFrame(train, columns=data.columns)
df_test = pd.DataFrame(test, columns=data.columns)

mu_lat = df_train['startPositionLat'].mean()
sig_lat = df_train['startPositionLat'].std()

mu_lng = df_train['startPositionLng'].mean()
sig_lng = df_train['startPositionLng'].std()

df_train['NormLng'] = df_train['startPositionLng'].apply(lambda x: (x-mu_lng)/sig_lng)
df_train['NormLat'] = df_train['startPositionLat'].apply(lambda x: (x-mu_lat)/sig_lat)


mu_cos = df_train['cos_timeOfDay'].mean()
sig_cos = df_train['cos_timeOfDay'].std()

mu_sin = df_train['sin_timeOfDay'].mean()
sig_sin = df_train['sin_timeOfDay'].std()

df_train['Normsin'] = df_train['sin_timeOfDay'].apply(lambda x: (x-mu_sin)/sig_sin)
df_train['Normcos'] = df_train['cos_timeOfDay'].apply(lambda x: (x-mu_cos)/sig_cos)


# Normalize validation set
test_mu_lat = df_test['startPositionLat'].mean()
test_sig_lat = df_test['startPositionLat'].std()

test_mu_lng = df_test['startPositionLng'].mean()
test_sig_lng = df_test['startPositionLng'].std()

df_test['NormLng'] = df_test['startPositionLng'].apply(lambda x: (x-test_mu_lng)/test_sig_lng)
df_test['NormLat'] = df_test['startPositionLat'].apply(lambda x: (x-test_mu_lat)/test_sig_lat)

test_mu_cos = df_test['cos_timeOfDay'].mean()
test_sig_cos = df_test['cos_timeOfDay'].std()

test_mu_sin = df_test['sin_timeOfDay'].mean()
test_sig_sin = df_test['sin_timeOfDay'].std()

df_test['Normsin'] = df_test['sin_timeOfDay'].apply(lambda x: (x-test_mu_sin)/test_sig_sin)
df_test['Normcos'] = df_test['cos_timeOfDay'].apply(lambda x: (x-test_mu_cos)/test_sig_cos)

data_columns = ['NormLng', 'NormLat', 'Normsin', 'Normcos']

In [10]:
data_train = df_train[['NormLng', 'NormLat', 'Normsin', 'Normcos']]
data_test = df_test[['NormLng', 'NormLat', 'Normsin', 'Normcos']]

# MODEL

In [11]:
class PriorConditioner_scaleFix(nn.Module):
    """
    This PyTorch Module implements the neural network used to condition the
    base distribution of a NF as in Eq.(13).
    """
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(PriorConditioner, self).__init__()

        #initialize linear transformations
        self.output_dim = output_dim
        self.input_dim = input_dim
        self.lin_input_to_hidden = nn.Linear(input_dim, hidden_dim)
        self.lin_hidden_to_hidden = nn.Linear(hidden_dim, hidden_dim)
        self.lin_hidden_to_loc = nn.Linear(hidden_dim, output_dim)
        self.lin_hidden_to_scale = nn.Linear(hidden_dim, output_dim)

        #initialize non-linearities
        self.relu = nn.ReLU()
        self.softplus = nn.Softplus()

    def forward(self, x):
        """
        Given input x=[z_{t-1}, h_t], this method outputs mean and 
        std.dev of the diagonal gaussian base distribution of a NF.
        """
        hidden = self.relu(self.lin_input_to_hidden(x))
        hidden = self.relu(self.lin_hidden_to_hidden(hidden))
        loc = self.lin_hidden_to_loc(hidden)
        scale = torch.ones_like(loc)
        return loc, scale

# VANILLA TIME CONDITIONED

In [13]:
mu = torch.zeros(4).to(device)
sigma = torch.ones(4).to(device)
base = Normal(mu, sigma)

In [14]:
net = RealNVP(in_features=4, prior=base, hidden_features=256, depth=12).to(device)
optimizer = torch.optim.Adam([p for p in net.parameters() if p.requires_grad], lr=1e-4)

pyro_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, patience=25, verbose=True)

# Arbitrary clipping value to suppress exploding gradients
clipping_value = 1

In [15]:
X_train = torch.Tensor(data_train.values).to(device)
X_test = torch.Tensor(data_test.values).to(device)

train_dataset = TensorDataset(X_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [16]:
train_losses = []
validation_losses = []
save_flag = True
epochs = 500

try:
    for epoch in tqdm(range(epochs)):
        loss_epoch = 0

        # VALIDATION
        net.eval()

        validation_loss = -torch.mean(net.log_likelihood(X_test))
        validation_losses.append(validation_loss.item())
        
        pyro_scheduler.step(validation_loss)

        # TRAIN LOOP
        net.train()
        for batch_num, (X_batch) in enumerate(train_loader):
            X_batch = X_batch[0]
            
            optimizer.zero_grad()
            
            loss = -torch.mean(net.log_likelihood(X_batch))
            
            loss.backward()
            optimizer.step()

            loss_epoch += loss.item()

        loss_epoch = loss_epoch / (batch_num+1)
        train_losses.append(loss_epoch)

        if (epoch) % 1 == 0:
            print(f'Validation loss: {validation_loss}')
            print(f'Train loss: {loss_epoch}')

        if (epoch) % 50 == 0:
            torch.save(net.state_dict(), 'model_state_fix_prior_time_cond_updated_epoch_%d.pth' % (epoch+1))
    
    if save_flag:
        loss_dict = {'validation': validation_losses, 'train': train_losses}
        with open('loss_dict_time_cond_fix_prior.pkl', 'wb') as f:
            pickle.dump(loss_dict, f)

        torch.save(net.state_dict(), 'model_state_fix_prior_cond.pth')

except KeyboardInterrupt:
    if save_flag:
        loss_dict = {'validation': validation_losses, 'train': train_losses}
        with open('loss_dict_time_cond_fix_prior.pkl', 'wb') as f:
            pickle.dump(loss_dict, f)

        torch.save(net.state_dict(), 'model_state_fix_prior_time_cond.pth')

  0%|          | 1/500 [00:21<2:55:04, 21.05s/it]

Validation loss: 1.6095812320709229
Train loss: -1.1750142817988398


  0%|          | 2/500 [00:41<2:54:14, 20.99s/it]

Validation loss: 0.7747608423233032
Train loss: -0.8280409979783814


  1%|          | 3/500 [01:03<2:54:49, 21.11s/it]

...

  3%|▎         | 15/500 [05:24<3:03:40, 22.72s/it]

Validation loss: -0.7395848035812378
Train loss: -4.670155524900674


  3%|▎         | 16/500 [05:49<3:08:06, 23.32s/it]

Validation loss: -3.414114475250244
Train loss: -4.729708460441274


  3%|▎         | 16/500 [06:09<3:06:12, 23.08s/it]
